In [1]:
# %%file stream_detection.py
import json
from kafka import KafkaConsumer
import joblib
import numpy as np

# Inicjalizacja Kafki
consumer = KafkaConsumer(
    'energy-usage',
    bootstrap_servers='broker:9092',
    value_deserializer=lambda m: json.loads(m.decode('utf-8')),
    group_id='anomaly-detection-group'
)

# Wczytanie modeli dla każdej gałęzi
models = {branch: joblib.load(f'isolation_forest_{branch}.joblib') for branch in ['Branch1', 'Branch2', 'Branch3', 'Branch4', 'Branch5']}

# Nasłuchiwanie wiadomości
for message in consumer:
    data = message.value
    branch = data['branch']
    energy_usage = data['energy_usage']
    timestamp = data['timestamp']
    
    # Sprawdzanie, czy dla danej gałęzi jest dostępny model
    if branch in models:
        # Konwersja danych do formatu oczekiwanego przez model
        input_data = np.array([[energy_usage]])
        prediction = models[branch].predict(input_data)
        
        # Wykrywanie anomalii
        if prediction[0] == -1:
            print(f"ALERT: Anomalous energy usage detected at {branch} with value {energy_usage} kWh at {timestamp}")
    else:
        print(f"No model available for branch {branch}, skipping prediction.")

ALERT: Anomalous energy usage detected at Branch1 with value 76.22 kWh at 2024-06-01 08:58:49.432127
ALERT: Anomalous energy usage detected at Branch1 with value 61.32 kWh at 2024-06-01 08:59:17.449249
ALERT: Anomalous energy usage detected at Branch4 with value 70.2 kWh at 2024-06-01 08:59:21.452408
ALERT: Anomalous energy usage detected at Branch5 with value 482.47 kWh at 2024-06-01 08:59:35.461970
ALERT: Anomalous energy usage detected at Branch4 with value 72.95 kWh at 2024-06-01 08:59:41.467277
ALERT: Anomalous energy usage detected at Branch3 with value 71.81 kWh at 2024-06-01 09:00:07.484999
ALERT: Anomalous energy usage detected at Branch2 with value 475.92 kWh at 2024-06-01 09:00:11.487074
ALERT: Anomalous energy usage detected at Branch2 with value 497.95 kWh at 2024-06-01 09:02:30.061621
ALERT: Anomalous energy usage detected at Branch1 with value 66.64 kWh at 2024-06-01 09:02:32.063591
ALERT: Anomalous energy usage detected at Branch3 with value 493.62 kWh at 2024-06-01 09:

KeyboardInterrupt: 